## UW Pharmacy Student Self-Care Analysis (spring 9_10)

This notebook is a draft using the "spring 9_10" page in the student comment data of the 2022-2023 SY in the UW SoP. The analysis roughly looks at the determinants of the categories of self-care students chose for this quarter using VADER and NLTK sentiment analysis and Pandas table manipulation.

Note: this notebook relies on uploading a single csv sheet for one quarter, along with adding two additional columns called "Category 1" and "Category 2" which categorize each comment based on the 8 facets of self-care.

### Setup

In [1]:
#import python libraries
import numpy as np
import pandas as pd

!pip install vaderSentiment
!pip install --upgrade pip
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

from textblob import TextBlob
import nltk
nltk.download('brown')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

import spacy

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
/shared-libs/python3.9/py/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-06-01 21:07:09.766761: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropria

In [2]:
# Can change the file csv right here
file_csv = "spring9_10.csv"

data = pd.read_csv(file_csv)

first_column = data.columns[1]
data = data[[first_column, "Category 1", "Category 2"]]

In [3]:
all_dups = pd.DataFrame()
for i in range(len(data)):
    if not pd.isnull(data["Category 2"][i]):
        temp = data[[first_column, "Category 2"]].iloc[i]
        all_dups = all_dups.append(temp)
    else:
        continue
all_dups = all_dups[[first_column, "Category 2"]]
all_dups = all_dups.rename(columns={"Category 2": "Category"})

### Working data

In [4]:
data = data.rename(columns={"Category 1": "Category"})
data = data[[first_column, "Category"]]
data = data.append(all_dups)
data

,Spr 9-10,Category
0,Set aside and spent some time at night after s...,Mental
1,I painted my nails! This is something I really...,Physical
2,"This weekend, I went shopping with my friend a...",Mental
3,I went out and had lunch with my brother.,Physical
4,It was a nice day today and my friend and I to...,Environmental
...,...,...
101,I am trying to revise for final and catch up b...,Emotional
104,Spent 20 minutes doing yoga,Spiritual
105,"This week, for health and wellness, I connecte...",Physical
106,I met a friend for lunch.,Community


### Category frequencies within student comments

The comments data is displayed here in terms of each of the 8 self-care categories (physical, mental, community, emotional, environmental, spiritual, and occupational) through a pie chart. 

In [5]:
import plotly.express as px

dfg = data.groupby("Category").count().sort_values(by=first_column, ascending=False)
print("All reflections processed (includes duplicate categories): ", np.sum(dfg[first_column]))

dfg_pie = px.pie(dfg.reset_index(), values='Spr 9-10', names='Category', title='Category Frequencies')
dfg_pie

All reflections processed (includes duplicate categories):  161


### Sentiment analysis (positivity polarity score for each comment)

In this section, I scored each of the comments using the VADER sentiment analysis function. The function sentiment_scores(sentence) will return the positivity polarity score for each comment probabilistically between -1 and 1, with -1 being the most negative and 1 being the most positive. Note that this model is untrained and will be a very rough interpretation of the comments, i.e. not trained on what self-care is as a whole but on the general tone of each comment.

In [6]:
# function to print sentiments
# of the sentence (vader)

#taken from geekforgeeks
def sentiment_scores(sentence):
 
    # Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()
 
    # polarity_scores method of SentimentIntensityAnalyzer
    # object gives a sentiment dictionary.
    # which contains pos, neg, neu, and compound scores.
    sentiment_dict = sid_obj.polarity_scores(sentence)

    return sentiment_dict['compound']

In [7]:
scores = pd.DataFrame(columns={"Win 1_2", "Score"})
for i in data[first_column]:
    sentence = str(i)
    score = sentiment_scores(sentence)
    scores = scores.append({first_column: sentence, "Score": score}, ignore_index=True)

In [8]:
scores_categories = data.merge(scores, on=first_column, how="right")
scores_categories

,Spr 9-10,Category,Score,Win 1_2
0,Set aside and spent some time at night after s...,Mental,0.0000,NaN
1,I painted my nails! This is something I really...,Physical,0.8404,NaN
2,"This weekend, I went shopping with my friend a...",Mental,0.7644,NaN
3,"This weekend, I went shopping with my friend a...",Community,0.7644,NaN
4,I went out and had lunch with my brother.,Physical,0.0000,NaN
...,...,...,...,...
283,"This week, for health and wellness, I connecte...",Physical,0.4404,NaN
284,I met a friend for lunch.,Physical,0.4939,NaN
285,I met a friend for lunch.,Community,0.4939,NaN
286,I started a new TV show with my partner,Mental,0.0000,NaN


In [9]:
fig = px.scatter(scores_categories, y="Score", x="Category", title="Polarity Scores by Category")
mean_scores = scores_categories.groupby('Category').mean()
for c in mean_scores.index:
    fig.add_scatter(x=[c],
                    y=[mean_scores.loc[c]['Score']],
                    marker=dict(
                        color='red',
                        size=10
                    ),
                name=f'{c} mean')

fig.show()

In [10]:
px.bar(mean_scores.reset_index().sort_values(by="Score", ascending=False), x="Category", y="Score", title="Average Scores per Category")

### Scores of each category 

In [11]:
print('Physical category table')
scores_categories_physical = scores_categories[scores_categories["Category"] == "Physical"]
scores_categories_physical

Physical category table


,Spr 9-10,Category,Score,Win 1_2
1,I painted my nails! This is something I really...,Physical,0.8404,NaN
4,I went out and had lunch with my brother.,Physical,0.0000,NaN
10,I walked for about 15-20 minutes around the ne...,Physical,0.2960,NaN
12,Took a nap,Physical,0.0000,NaN
18,This morning I decided to walk to campus inste...,Physical,0.7517,NaN
...,...,...,...,...
268,I took a walk outside.,Physical,0.0000,NaN
277,I took a walk around the neighborhood to liste...,Physical,0.7506,NaN
280,Spent 20 minutes doing yoga,Physical,0.0000,NaN
283,"This week, for health and wellness, I connecte...",Physical,0.4404,NaN


In [12]:
print('Mental category table')
scores_categories_mental = scores_categories[scores_categories["Category"] == "Mental"]
scores_categories_mental

Mental category table


,Spr 9-10,Category,Score,Win 1_2
0,Set aside and spent some time at night after s...,Mental,0.0000,NaN
2,"This weekend, I went shopping with my friend a...",Mental,0.7644,NaN
16,Last week my D&D group was finally able to syn...,Mental,0.9845,NaN
46,I went furniture shopping with my husband.,Mental,0.0000,NaN
59,Today I had to step away from everything schoo...,Mental,0.5984,NaN
64,Knitted while watching youtube video.,Mental,0.0000,NaN
77,Reading from my current book (A People's Histo...,Mental,0.4215,NaN
78,Made a pasta Greek salad.,Mental,0.0000,NaN
93,"I was so lucky, I got to go to the Woodland Pa...",Mental,0.9863,NaN
102,I watched the new Dr. Strange movie with a gro...,Mental,0.3182,NaN


In [13]:
print('Community category table')
scores_categories_community = scores_categories[scores_categories["Category"] == "Community"]
scores_categories_community

Community category table


,Spr 9-10,Category,Score,Win 1_2
3,"This weekend, I went shopping with my friend a...",Community,0.7644,NaN
5,I went out and had lunch with my brother.,Community,0.0000,NaN
7,It was a nice day today and my friend and I to...,Community,0.9633,NaN
15,Last week my D&D group was finally able to syn...,Community,0.9845,NaN
23,I visited the dunes near Moses Lake with my fr...,Community,0.5255,NaN
24,I found it really hard to do self care this we...,Community,0.5722,NaN
30,We had a little party at Optimism Brewing to c...,Community,0.9252,NaN
36,The highlight of the week was the welcome to t...,Community,0.9413,NaN
47,I went furniture shopping with my husband.,Community,0.0000,NaN
48,We had a family picnic !,Community,0.0000,NaN


In [14]:
print('Emotional category table')
scores_categories_emotional = scores_categories[scores_categories["Category"] == "Emotional"]
scores_categories_emotional

Emotional category table


,Spr 9-10,Category,Score,Win 1_2
25,I found it really hard to do self care this we...,Emotional,0.5722,NaN
26,I listened to soothing music for 30 minutes,Emotional,0.3182,NaN
37,Our PY1 year is coming to an end and I have a ...,Emotional,0.9980,NaN
45,"Spring is here. Today, I would like to spend a...",Emotional,0.9862,NaN
52,I have been trying to journal more to get my t...,Emotional,0.9258,NaN
60,I decided to listen to music for a 30 min brea...,Emotional,0.9118,NaN
65,I made a motivation board to get me through fi...,Emotional,-0.4496,NaN
98,"Today, I listened to music while taking a walk...",Emotional,0.0000,NaN
126,As the last week of our first year of Pharmacy...,Emotional,0.9147,NaN
141,I listened to music.,Emotional,0.0000,NaN


In [15]:
print('Spiritual category table')
scores_categories_spiritual = scores_categories[scores_categories["Category"] == "Spiritual"]
scores_categories_spiritual

Spiritual category table


,Spr 9-10,Category,Score,Win 1_2
80,I was so overwhelmed during the last couple we...,Spiritual,0.9084,NaN
95,I took 30 minutes today to walk by the water a...,Spiritual,0.0000,NaN
150,I meditated,Spiritual,0.0000,NaN
161,Spent 20 minutes doing yoga,Spiritual,0.0000,NaN
223,I was so overwhelmed during the last couple we...,Spiritual,0.9084,NaN
233,I took 30 minutes today to walk by the water a...,Spiritual,0.0000,NaN
281,Spent 20 minutes doing yoga,Spiritual,0.0000,NaN


In [16]:
print('Occupational category table')
scores_categories_occupational = scores_categories[scores_categories["Category"] == "Occupational"]
scores_categories_occupational

Occupational category table


,Spr 9-10,Category,Score,Win 1_2
20,Regrouped myself and spent 15 minutes organizi...,Occupational,0.0000,NaN
32,I have been studying and working hard to get a...,Occupational,0.7263,NaN
62,I’m studying for finals :(.,Occupational,0.0000,NaN
71,I went to a coffee shop to study today.,Occupational,0.0000,NaN
118,I went to a coffee shop to study today :),Occupational,0.4588,NaN
121,Went to a coffee shop to study for finals.,Occupational,0.0000,NaN
137,We have a busy schedule this week as it gets c...,Occupational,0.6907,NaN
149,"As finals week is coming up, I have been takin...",Occupational,0.8265,NaN
156,I am trying to revise for final and catch up b...,Occupational,0.0000,NaN
217,I went to a coffee shop to study today.,Occupational,0.0000,NaN


### Frequency of nouns and verbs 

In [17]:
!pip install spacy -q
!python -m spacy download en_core_web_sm -q

from collections import Counter
import en_core_web_sm

2023-06-01 21:07:22.174933: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-01 21:07:22.355509: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-01 21:07:22.355549: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-06-01 21:07:22.391285: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-01 21:07:23.251252: W tensorflow/stream_executor/pla

Note: Just for the sake of experimentation, I only did this part on comments categorized as "physical." Can easily broaden it to more categories if useful.

This section takes the top 100 ('top' meaning most frequent) words from a particular category along with its count within the comments. Again, note that there may be duplicates depending on if the comment was repeated if it was categorized twice. I then created a function to extract nouns and verbs within that category. This can potentially be helpful if we were to see which action within each category students tended to choose.

In [18]:
def fig_frequencies(tbl, keyword):
    counter = Counter(" ".join(tbl[first_column]).split()).most_common(100)
    array = []
    for c in counter:
        array = np.append(array, c[0])

    sentence = ','.join(array)

    def extract_nouns_verbs(sentence):
        text = nltk.word_tokenize(sentence)
        pos_tagged = nltk.pos_tag(text)
        nouns_verbs = filter(lambda x:x[1]=='NN' or x[1] == 'VB',pos_tagged)
        return list(nouns_verbs)

    nouns_verbs = pd.DataFrame(extract_nouns_verbs(sentence), columns=['Word', 'Word type'])
    counted_words = pd.DataFrame(counter, columns=['Word', 'Word frequency'])

    noun_verb_frequency = nouns_verbs.merge(counted_words, on='Word', how='left').sort_values(by='Word frequency', ascending=False)

    nlp = en_core_web_sm.load()
    def all_nouns(sentence):
        doc = nlp(sentence)
        nouns = [(token.lemma_, "NN") for token in doc if token.pos_ == "NOUN"]
        return nouns

    def all_verbs(sentence):
        doc = nlp(sentence)
        verbs = [(token.lemma_, "VB") for token in doc if token.pos_ == "VERB"]
        return verbs

    nouns_verbs_spacy = all_nouns(sentence) + all_verbs(sentence)

    nouns_verbs_sp = pd.DataFrame(nouns_verbs_spacy, columns=['Word', 'Word type'])
    counted_words = pd.DataFrame(counter, columns=['Word', 'Word frequency'])

    noun_verb_frequency_sp = nouns_verbs_sp.merge(counted_words, on='Word', how='inner').sort_values(by='Word frequency', ascending=False)
    noun_verb_frequency_sp = noun_verb_frequency_sp.drop_duplicates(subset='Word', keep="last")

    fig = px.bar(noun_verb_frequency_sp, x='Word', y='Word frequency', color='Word type', title='Noun/verb frequencies for Category: ' + keyword)
    fig.update_layout(xaxis_categoryorder = 'total descending')

    fig.show()

In [19]:
fig_frequencies(scores_categories_physical, 'Physical');

/shared-libs/python3.9/py/lib/python3.9/site-packages/spacy/language.py:1895: UserWarning:

[W123] Argument disable with value [] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.



In [20]:
fig_frequencies(scores_categories_mental, 'Mental');

In [21]:
fig_frequencies(scores_categories_emotional, 'Emotional');

In [22]:
fig_frequencies(scores_categories_community, 'Community');

In [23]:
fig_frequencies(scores_categories_spiritual, 'Spiritual');

In [24]:
fig_frequencies(scores_categories_occupational, 'Occupational');

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=443b2d8b-ed93-43a6-bbe3-c3fe69ac80e1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>